In [1]:
### Colab define cell
FOLDER_PATH = '/content/gdrive/MyDrive/Counterfactual-prototype-main/LORE/LORE-master'

try:
    import google.colab
    RUNNING_ON_COLAB = True
except: 
    RUNNING_ON_COLAB = False

if RUNNING_ON_COLAB: 
    import sys
    from google.colab import drive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials 
    drive.mount('/content/gdrive', force_remount=True)
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    gdrive = GoogleDrive(gauth)
    sys.path.append(FOLDER_PATH)

In [2]:
from utils.prediction import PredictionTypeWrapper, PredictionTypeWrapper, generate_local_predictions, wrap_information
from utils.cf import generate_cf_for_all
from utils import load
from alibi_cf import AlibiCounterfactaulWrapper
from alibi.explainers import CounterFactualProto, CounterFactual
import dice_ml
import numpy as np
import pandas as pd

import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs

### Set random seed
seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)

import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
### Load data
dataset = load.SelectableDataset.Diabetes
data, balanced_data, X, Y, encoder, scaler, n_features, n_classes, feature_names, target_name = load.load_dataset(dataset)
X_train, Y_train, X_test, Y_test, X_validation, Y_validation = load.load_training_data(dataset)
model = load.load_trained_model_for_dataset(dataset)
input_data_shape = np.array([X[0]]).shape

Loaded model from disk


In [4]:
diabetes_feature_range = (X_train.min(axis=0), X_train.max(axis=0))

local_data_dict = generate_local_predictions( X_test, Y_test, model, scaler, encoder )
true_positives,true_negatives, false_positives, false_negatives = wrap_information( local_data_dict )
all_predictions = PredictionTypeWrapper(true_positives,true_negatives, false_positives, false_negatives)

/Users/jrhs/miniforge3/envs/tf_mac/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [5]:
watcher_cf = CounterFactual(model, input_data_shape, distance_fn='l1', target_proba=1.0,
                    target_class='other', max_iter=3000, early_stop=150, lam_init=1e-1,
                    max_lam_steps=10, tol=0.05, learning_rate_init=0.1,
                    feature_range=diabetes_feature_range, eps=0.01, init='identity',
                    decay=True, write_dir=None, debug=False)

cf_p = CounterFactualProto(model, input_data_shape, use_kdtree=True, theta=10., max_iterations=1000,
                         feature_range=diabetes_feature_range,
                         c_init=1., c_steps=10)

cf_p.fit(X_train)

""

''

In [6]:
alibi_cf =  AlibiCounterfactaulWrapper(cf_p, watcher_cf, scaler, feature_names)

In [ ]:
watcher_result_df = generate_cf_for_all(
    all_predictions,
    alibi_cf.watcher_counterfactual_explain,
    feature_names
    )


====================Doing TruePositive====================
|  

====================Running...====================
| Instance 0 

====================Running...====================
| Instance 1 


In [ ]:
watcher_result_df.to_csv("watcher_cf_result.csv")

In [7]:
proto_result_df = generate_cf_for_all(
    all_predictions,
    alibi_cf.counterfactual_proto_explain,
    feature_names
    )


====================Doing TruePositive====================
|  

====================Running...====================
| Instance 0 

====================Running...====================
| Instance 1 

====================Running...====================
| Instance 2 

====================Running...====================
| Instance 3 

====================Running...====================
| Instance 4 

====================Running...====================
| Instance 5 

====================Running...====================
| Instance 6 

====================Running...====================
| Instance 7 

====================Running...====================
| Instance 8 

====================Running...====================
| Instance 9 

====================Running...====================
| Instance 10 

====================Running...====================
| Instance 11 

====================Running...====================
| Instance 12 

====================Running...====================
| Instance 13 

=======

In [8]:
proto_result_df.to_csv("cf_proto_result.csv")